In [ ]:
import pandas as pd

In [ ]:
#Cargamos el archivo a utilizar 
events = pd.read_csv("data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

# Funciones útiles

In [ ]:
quincenas = ['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']

def aplicar_por_quincena(train, funcion):
    
    for quincena in quincenas:
        train = funcion(train, quincena)
    return train

# Feature quincenas

In [ ]:
events['timestamp'] = pd.to_datetime(events['timestamp'])
events['mes'] = events['timestamp'].dt.month
events['dia'] = events['timestamp'].dt.day

In [ ]:
events['q1'] = ((events['mes']==1) & (events['dia']>1) & (events['dia']<16))
events['q2'] = ((events['mes']==1) & (events['dia']>15) & (events['dia']<32))
events['q3'] = ((events['mes']==2) & (events['dia']>1) & (events['dia']<16))
events['q4'] = ((events['mes']==2) & (events['dia']>15) & (events['dia']<29))
events['q5'] = ((events['mes']==3) & (events['dia']>1) & (events['dia']<16))
events['q6'] = ((events['mes']==3) & (events['dia']>15) & (events['dia']<32))
events['q7'] = ((events['mes']==4) & (events['dia']>1) & (events['dia']<16))
events['q8'] = ((events['mes']==4) & (events['dia']>15) & (events['dia']<31))
events['q9'] = ((events['mes']==5) & (events['dia']>1) & (events['dia']<16))
events['q10'] = ((events['mes']==5) & (events['dia']>15) & (events['dia']<32))

# Feature cantidad de eventos por persona

In [ ]:
events = events.join(events.groupby('person').event.count(), on='person', rsuffix='_count')

# Feature tiempo entre primer y última entrada al sitio por persona

In [ ]:
events = events.join(events.groupby('person').timestamp.agg('max') - events.groupby('person').timestamp.agg('min'), on='person', rsuffix='_total')
events = events.rename(columns={'timestamp_total' : 'tiempo_total'})

In [ ]:
events.columns

In [ ]:
pd.DataFrame(events.tiempo_total.unique()).head(10)

# Feature cantidad de events por persona por quincena

In [ ]:
def cantidad_eventos_por_persona(df, quincena):
    df = df.join(df.loc[df[quincena] == True].groupby('person').event.count(), on='person', rsuffix=('_count_'+quincena))
    return df

In [ ]:
events = (aplicar_por_quincena(events, cantidad_eventos_por_persona))
events.columns

# Feature tiempo entre primer y última entrada al sitio en quincena por persona

In [ ]:
def tiempo_total_por_persona(df, quincena):
    df_quincena = df.loc[df[quincena] == True]
    df_quincena = df.join(df_quincena.groupby('person').timestamp.agg(lambda x: (x.max() - x.min())), on='person', rsuffix='_total_'+quincena)
    df_quincena = df_quincena.rename(columns={'timestamp_total_'+quincena : 'tiempo_total_'+quincena})
    
    return df_quincena

In [ ]:
events = aplicar_por_quincena(events, tiempo_total_por_persona)
events.columns

In [ ]:
pd.DataFrame(events.tiempo_total_q1.unique()).head(10)

# Export a csv

In [ ]:
events.to_csv('data/events_with_features.csv', index=False)